# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine, text

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def make_conn(**db_parameters):
    """
    Composes a SQLAlchemy connect string from the given database connection parameters.

    https://github.com/snowflakedb/snowflake-sqlalchemy#escaping-special-characters-such-as---signs-in-passwords
    
    """
    try:
        engine = create_engine(URL(**db_parameters))
        return engine.connect()
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    
    finally:
        engine.connect().close()
        engine.dispose()

# Establsih Connection

In [3]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database='', schema='', warehouse=WAREHOUSE)

# Create Database and Schema

Create Database

In [4]:
db_name = "aws_metro_bike_share"

create_database_query = f"""
                         CREATE DATABASE IF NOT EXISTS {db_name}
                         """

conn.execute(text(create_database_query))

Create Schema

In [5]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database=db_name, schema='', warehouse=WAREHOUSE)

schema_name = "aws_metro_bike_share_schema"

create_schema_query = f"""
                       CREATE SCHEMA IF NOT EXISTS {schema_name}
                       """

conn.execute(text(create_schema_query))

# Create AWS Integration

In [6]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database=db_name, schema=schema_name, warehouse=WAREHOUSE)
integration_name = 'aws_s3_integration'
integration_comment = 'AWS Storage Integration'

aws_integration_query = f"""
                         CREATE OR REPLACE STORAGE INTEGRATION {integration_name}
                         TYPE = EXTERNAL_STAGE
                         STORAGE_PROVIDER = 'S3'
                         ENABLED = TRUE
                         STORAGE_AWS_ROLE_ARN = '{AWS_ARN}'
                         STORAGE_ALLOWED_LOCATIONS = ('{AWS_LOCATION}')
                         COMMENT = '{integration_comment}'

                        """

conn.execute(text(aws_integration_query))
conn.execute(text("""SHOW INTEGRATIONS like '%s3%'""")).fetchall()

[('AWS_S3_INTEGRATION', 'EXTERNAL_STAGE', 'STORAGE', 'true', 'AWS Storage Integration', datetime.datetime(2023, 7, 24, 15, 44, 5, 724000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>))]

In [7]:
grant_access_query = f"""
                      GRANT USAGE ON INTEGRATION {integration_name} TO ROLE {ROLE}
                      """

conn.execute(text(grant_access_query)).fetchall()

[('Statement executed successfully.',)]

# Specify File Format

In [8]:
file_format_name = "my_csv_file"
file_type = "CSV"

file_format_query = f"""
                     CREATE OR REPLACE FILE FORMAT {file_format_name}
                     TYPE = {file_type}
                     """

conn.execute(text(file_format_query)).fetchall()

[('File format MY_CSV_FILE successfully created.',)]

# Create Stage

In [9]:
aws_stage_name = "aws_s3_stage"

aws_stage_query = f"""
                   CREATE OR REPLACE STAGE {aws_stage_name}
                   STORAGE_INTEGRATION = {integration_name}
                   FILE_FORMAT = {file_format_name}
                   URL = {AWS_LOCATION}
                   """

conn.execute(text(aws_stage_query)).fetchall()

[('Stage area AWS_S3_STAGE successfully created.',)]

# Create Table

In [10]:
table_name = "aws_bikeshare"
table_comment = "my bike_share snowflake aws table"

create_table_query = f"""CREATE TABLE IF NOT EXISTS {table_name} (
                                                                   trip_id VARCHAR,
                                                                   duration VARCHAR,
                                                                   start_time VARCHAR,
                                                                   end_time VARCHAR,
                                                                   start_station VARCHAR,
                                                                   start_lat VARCHAR,
                                                                   start_lon VARCHAR,
                                                                   end_station VARCHAR,
                                                                   end_lat VARCHAR,
                                                                   end_lon VARCHAR,
                                                                   bike_id VARCHAR,
                                                                   plan_duration VARCHAR,
                                                                   trip_route_category STRING,
                                                                   passholder_type STRING,
                                                                   bike_type STRING
                                                                )
                         
                                    COMMENT = '{table_comment}'
                                """

conn.execute(text(create_table_query)).fetchall()

[('Table AWS_BIKESHARE successfully created.',)]

# Copy Data Into Table

In [11]:
copy_data_query = f"""
                   COPY INTO {table_name}
                   FROM @{aws_stage_name}
                   FILE_FORMAT = (TYPE = CSV)
                   ON_ERROR = ABORT_STATEMENT
                   PURGE = FALSE
        
                   """

conn.execute(text(copy_data_query)).fetchall()

C:\Users\ASANTE\AppData\Local\Temp\ipykernel_21628\2767545508.py:10: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(text(copy_data_query)).fetchall()


[('s3://bikeshare-2023/metro-trips-2023-q2.csv', 'LOADED', 109436, 109436, 1, 0, None, None, None, None)]

# Query Table

In [12]:
data = conn.execute(text(f"""SELECT DISTINCT * FROM {table_name}""")).fetchall()
df = pd.DataFrame(data)

# View DataFrame

In [13]:
df.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,249955060,41,4/1/2023 0:00,4/1/2023 0:41,4643,34.07262,-118.44944,4643,34.07262,-118.44944,22398,30,Round Trip,Monthly Pass,electric
1,249954667,2,4/1/2023 0:02,4/1/2023 0:04,4523,34.068748,-118.30928,4538,34.06543,-118.308823,6591,30,One Way,Monthly Pass,standard
2,249954776,12,4/1/2023 0:03,4/1/2023 0:15,3034,34.042061,-118.263382,4491,34.04744,-118.24794,13897,365,One Way,Annual Pass,standard
3,249954777,11,4/1/2023 0:04,4/1/2023 0:15,3056,34.03746,-118.265381,4491,34.04744,-118.24794,15488,30,One Way,Monthly Pass,standard
4,249954826,15,4/1/2023 0:04,4/1/2023 0:19,4538,34.06543,-118.308823,4538,34.06543,-118.308823,20027,30,Round Trip,Monthly Pass,standard


In [14]:
df.tail()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
109431,276550460,25,6/30/2023 19:49,6/30/2023 20:14,4266,34.026291,-118.277687,4405,34.051899,-118.293678,25355,30,One Way,Monthly Pass,electric
109432,276549639,4,6/30/2023 19:56,6/30/2023 20:00,4303,34.098011,-118.287071,4472,34.092602,-118.28093,6020,365,One Way,Annual Pass,standard
109433,276552494,7,6/30/2023 20:40,6/30/2023 20:47,4214,33.99556,-118.481552,4207,34.000881,-118.46891,6617,365,One Way,Annual Pass,standard
109434,276565577,7,6/30/2023 21:31,6/30/2023 21:38,3018,34.043732,-118.260139,3008,34.046612,-118.262733,24830,30,One Way,Monthly Pass,electric
109435,276590585,23,6/30/2023 23:44,7/1/2023 0:07,3049,34.056969,-118.253593,3049,34.056969,-118.253593,6205,1,Round Trip,Walk-up,standard
